## Hardware check

In [1]:
# gpu check
!nvidia-smi

Fri Jun 14 08:39:00 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM2-32GB           On  |   00000000:3A:00.0 Off |                    0 |
| N/A   30C    P0             41W /  300W |      17MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# check number of cores
import multiprocessing

cores = multiprocessing.cpu_count() 
cores

80

## Environment Prep

In [3]:
cd /pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis/

/pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis


In [4]:
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Lambda, GlobalAveragePooling1D
# from tensorflow.keras.models import Model
from tensorflow.keras import models
import pickle
import numpy as np
from tqdm import tqdm
tqdm.pandas()
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback, EarlyStopping
import pandas as pd
import json
from torch.utils.data import Dataset
from transformers import AutoTokenizer, pipeline, AutoModel
import resources.smart_cond as sc
# from google.colab import files
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
import os

2024-06-14 08:39:00.868235: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-14 08:39:00.917605: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-14 08:39:02.048696: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
train_path = 'CLS/GloVe/Data/classification_train.txt'
train_text = []

with open(train_path) as file:
    for line in tqdm(file):
        train_text.append(line.rstrip())

# train_text = np.array(train_text)

36551it [00:00, 63485.61it/s]


In [6]:
valid_path = 'CLS/GloVe/Data/classification_valid.txt'
valid_text = []

with open(valid_path) as file:
    for line in tqdm(file):
        valid_text.append(line.rstrip())

# train_text = np.array(train_text)

9262it [00:00, 53674.09it/s]


In [7]:
test_path = 'CLS/GloVe/Data/classification_test.txt'
test_text = []

with open(test_path) as file:
    for line in tqdm(file):
        test_text.append(line.rstrip())

# train_text = np.array(train_text)

11469it [00:00, 69753.32it/s]


In [8]:
# prepare tokenizer
t = Tokenizer()
train_val_ip = train_text+valid_text
t.fit_on_texts(train_val_ip)
# vocab_size = len(t.word_index) + 1
vocab_size = 297321
print(f'vocabulary size: {vocab_size}')
del train_val_ip

# encode
encoded_train = t.texts_to_sequences(train_text)
encoded_valid = t.texts_to_sequences(valid_text)
encoded_test = t.texts_to_sequences(test_text)


# pad to max_len
max_length = 24135
padded_train = pad_sequences(encoded_train, maxlen=max_length, padding='post')
padded_valid = pad_sequences(encoded_valid, maxlen=max_length, padding='post')
padded_test = pad_sequences(encoded_test, maxlen=max_length, padding='post')

vocabulary size: 297321


In [9]:
data_path = 'Data/CLS/cls_data.pkl'
train_ip, valid_ip, test_ip, train_op, valid_op, test_op = pickle.load(open(data_path, 'rb'))
train_ip = train_ip[:4]
valid_ip = valid_ip[:4]
test_ip = test_ip[:4]

In [10]:
train_ip.append(padded_train)
valid_ip.append(padded_valid)
test_ip.append(padded_test)

In [11]:
from numpy import array, asarray, zeros

# GloVe
glove_path = 'resources/glove.840B.300d.txt'

embedding_model = {}
f = open('resources/glove.840B.300d.txt', 'r', encoding="utf8")
for line in tqdm(f):
    values = line.split()
    word = ''.join(values[:-300])
    coefs = np.asarray(values[-300:], dtype='float32')
    embedding_model[word] = coefs
f.close()

2196017it [01:29, 24490.23it/s]


In [12]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 300))
for word, i in tqdm(t.word_index.items()):
    embedding_vector = embedding_model.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 289274/289274 [00:00<00:00, 551466.81it/s]


In [13]:
def get_res(y_true, y_pred):
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred)
    pr_auc = auc(recall, precision)
    minrp = np.minimum(precision, recall).max()
    roc_auc = roc_auc_score(y_true, y_pred)
    return [roc_auc, pr_auc, minrp]

######################################################################################################## 
######################################################################################################## 
class_weights = compute_class_weight(class_weight='balanced', classes=[0,1], y=train_op)
def mortality_loss(y_true, y_pred):
    sample_weights = (1-y_true)*class_weights[0] + y_true*class_weights[1]
    bce = K.binary_crossentropy(y_true, y_pred)
    return K.mean(sample_weights*bce, axis=-1)
######################################################################################################## 
######################################################################################################## 

# var_weights = np.sum(fore_train_op[:, V:], axis=0)
# var_weights[var_weights==0] = var_weights.max()
# var_weights = var_weights.max()/var_weights
# var_weights = var_weights.reshape((1, V))
def forecast_loss(y_true, y_pred):
    return K.sum(y_true[:,V:]*(y_true[:,:V]-y_pred)**2, axis=-1)

def get_min_loss(weight):
    def min_loss(y_true, y_pred):
        return weight*y_pred
    return min_loss

class CustomCallback(Callback):
    def __init__(self, validation_data, batch_size):
        self.val_x, self.val_y = validation_data
        self.batch_size = batch_size
        super(Callback, self).__init__()

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.val_x, verbose=0, batch_size=self.batch_size)
        if type(y_pred)==type([]):
            y_pred = y_pred[0]
        precision, recall, thresholds = precision_recall_curve(self.val_y, y_pred)
        pr_auc = auc(recall, precision)
        roc_auc = roc_auc_score(self.val_y, y_pred)
        logs['custom_metric'] = pr_auc + roc_auc
        print ('val_aucs:', pr_auc, roc_auc)

In [14]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Embedding, Activation, Dropout, Softmax, Layer, InputSpec, Input, Dense, Lambda, TimeDistributed, Concatenate, Add
from tensorflow.keras import initializers, regularizers, constraints, Model
from tensorflow.python.keras.utils import tf_utils
from tensorflow.python.ops import array_ops
from tensorflow import nn

    
class CVE(Layer):
    def __init__(self, hid_units, output_dim):
        self.hid_units = hid_units
        self.output_dim = output_dim
        super(CVE, self).__init__()
        
    def build(self, input_shape): 
        self.W1 = self.add_weight(name='CVE_W1',
                            shape=(1, self.hid_units),
                            initializer='glorot_uniform',
                            trainable=True)
        self.b1 = self.add_weight(name='CVE_b1',
                            shape=(self.hid_units,),
                            initializer='zeros',
                            trainable=True)
        self.W2 = self.add_weight(name='CVE_W2',
                            shape=(self.hid_units, self.output_dim),
                            initializer='glorot_uniform',
                            trainable=True)
        super(CVE, self).build(input_shape)
        
    def call(self, x):
        x = K.expand_dims(x, axis=-1)
        x = K.dot(K.tanh(K.bias_add(K.dot(x, self.W1), self.b1)), self.W2)
        return x
        
    def compute_output_shape(self, input_shape):
        return input_shape + (self.output_dim,)
    
    
class Attention(Layer):
    
    def __init__(self, hid_dim):
        self.hid_dim = hid_dim
        super(Attention, self).__init__()

    def build(self, input_shape):
        d = input_shape.as_list()[-1]
        self.W = self.add_weight(shape=(d, self.hid_dim), name='Att_W',
                                 initializer='glorot_uniform',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.hid_dim,), name='Att_b',
                                 initializer='zeros',
                                 trainable=True)
        self.u = self.add_weight(shape=(self.hid_dim,1), name='Att_u',
                                 initializer='glorot_uniform',
                                 trainable=True)
        super(Attention, self).build(input_shape)
        
    def call(self, x, mask, mask_value=-1e30):
        attn_weights = K.dot(K.tanh(K.bias_add(K.dot(x,self.W), self.b)), self.u)
        mask = K.expand_dims(mask, axis=-1)
        attn_weights = mask*attn_weights + (1-mask)*mask_value
        attn_weights = K.softmax(attn_weights, axis=-2)
        return attn_weights
        
    def compute_output_shape(self, input_shape):
        return input_shape[:-1] + (1,)
    
    
class Transformer(Layer):
    
    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0):
        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout
        self.epsilon = K.epsilon() * K.epsilon()
        super(Transformer, self).__init__()

    def build(self, input_shape):
        d = input_shape.as_list()[-1]
        if self.dk==None:
            self.dk = d//self.h
        if self.dv==None:
            self.dv = d//self.h
        if self.dff==None:
            self.dff = 2*d
        self.Wq = self.add_weight(shape=(self.N, self.h, d, self.dk), name='Wq',
                                 initializer='glorot_uniform', trainable=True)
        self.Wk = self.add_weight(shape=(self.N, self.h, d, self.dk), name='Wk',
                                 initializer='glorot_uniform', trainable=True)
        self.Wv = self.add_weight(shape=(self.N, self.h, d, self.dv), name='Wv',
                                 initializer='glorot_uniform', trainable=True)
        self.Wo = self.add_weight(shape=(self.N, self.dv*self.h, d), name='Wo',
                                 initializer='glorot_uniform', trainable=True)
        self.W1 = self.add_weight(shape=(self.N, d, self.dff), name='W1',
                                 initializer='glorot_uniform', trainable=True)
        self.b1 = self.add_weight(shape=(self.N, self.dff), name='b1',
                                 initializer='zeros', trainable=True)
        self.W2 = self.add_weight(shape=(self.N, self.dff, d), name='W2',
                                 initializer='glorot_uniform', trainable=True)
        self.b2 = self.add_weight(shape=(self.N, d), name='b2',
                                 initializer='zeros', trainable=True)
        self.gamma = self.add_weight(shape=(2*self.N,), name='gamma',
                                 initializer='ones', trainable=True)
        self.beta = self.add_weight(shape=(2*self.N,), name='beta',
                                 initializer='zeros', trainable=True)
        super(Transformer, self).build(input_shape)
        
    def call(self, x, mask, mask_value=-1e-30):
        mask = K.expand_dims(mask, axis=-2)
        for i in range(self.N):
            # MHA
            mha_ops = []
            for j in range(self.h):
                q = K.dot(x, self.Wq[i,j,:,:])
                k = K.permute_dimensions(K.dot(x, self.Wk[i,j,:,:]), (0,2,1))
                v = K.dot(x, self.Wv[i,j,:,:])
                A = K.batch_dot(q,k)
                # Mask unobserved steps.
                A = mask*A + (1-mask)*mask_value
                # Mask for attention dropout.
                def dropped_A():
                    dp_mask = K.cast((K.random_uniform(shape=array_ops.shape(A))>=self.dropout), K.floatx())
                    return A*dp_mask + (1-dp_mask)*mask_value
                A = sc.smart_cond(K.learning_phase(), dropped_A, lambda: array_ops.identity(A))
                A = K.softmax(A, axis=-1)
                mha_ops.append(K.batch_dot(A,v))
            conc = K.concatenate(mha_ops, axis=-1)
            proj = K.dot(conc, self.Wo[i,:,:])
            # Dropout.
            proj = sc.smart_cond(K.learning_phase(), lambda: array_ops.identity(nn.dropout(proj, rate=self.dropout)),\
                                       lambda: array_ops.identity(proj))
            # Add & LN
            x = x+proj
            mean = K.mean(x, axis=-1, keepdims=True)
            variance = K.mean(K.square(x - mean), axis=-1, keepdims=True)
            std = K.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x*self.gamma[2*i] + self.beta[2*i]
            # FFN
            ffn_op = K.bias_add(K.dot(K.relu(K.bias_add(K.dot(x, self.W1[i,:,:]), self.b1[i,:])), 
                           self.W2[i,:,:]), self.b2[i,:,])
            # Dropout.
            ffn_op = sc.smart_cond(K.learning_phase(), lambda: array_ops.identity(nn.dropout(ffn_op, rate=self.dropout)),\
                                       lambda: array_ops.identity(ffn_op))
            # Add & LN
            x = x+ffn_op
            mean = K.mean(x, axis=-1, keepdims=True)
            variance = K.mean(K.square(x - mean), axis=-1, keepdims=True)
            std = K.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x*self.gamma[2*i+1] + self.beta[2*i+1]            
        return x
        
    def compute_output_shape(self, input_shape):
        return input_shape


def build_strats(D, max_len, V, d, N, he, dropout, forecast=False):
    demo = Input(shape=(D,))
    demo_enc = Dense(2*d, activation='tanh')(demo)
    demo_enc = Dense(d, activation='tanh')(demo_enc)
    varis = Input(shape=(max_len,))
    values = Input(shape=(max_len,))
    times = Input(shape=(max_len,))
    varis_emb = Embedding(V+1, d)(varis)
    cve_units = int(np.sqrt(d))
    values_emb = CVE(cve_units, d)(values)
    times_emb = CVE(cve_units, d)(times)
    comb_emb = Add()([varis_emb, values_emb, times_emb]) # b, L, d
#     demo_enc = Lambda(lambda x:K.expand_dims(x, axis=-2))(demo_enc) # b, 1, d
#     comb_emb = Concatenate(axis=-2)([demo_enc, comb_emb]) # b, L+1, d
    mask = Lambda(lambda x:K.clip(x,0,1))(varis) # b, L
#     mask = Lambda(lambda x:K.concatenate((K.ones_like(x)[:,0:1], x), axis=-1))(mask) # b, L+1
    cont_emb = Transformer(N, he, dk=None, dv=None, dff=None, dropout=dropout)(comb_emb, mask=mask)
    attn_weights = Attention(2*d)(cont_emb, mask=mask)
    fused_emb = Lambda(lambda x:K.sum(x[0]*x[1], axis=-2))([cont_emb, attn_weights])
    
    
    ## GloVe text embedding starts here
    max_length = 24135
    # embed text input
    texts = Input(shape=(max_length,))
    embedded_text = Embedding(vocab_size, 300, weights=[
    embedding_matrix], input_length=max_length, trainable=False)(texts)
    pooled = GlobalAveragePooling1D()(embedded_text)
    # flattened = Flatten()(embedded_text)
    
    text_0 = Dense(100, activation='relu')(pooled)
    
    
    # hidden_states = LSTM(64, return_sequences=True, name='lstm_layer')(embedded_text)
    # pooled = AveragePooling1D()(hidden_states)
    # text_0 = Dense(100, activation='relu')(hidden_states)
    
    # text_1 = Flatten()(hidden_states)
    
    # text_enc = Dense(22528, activation='relu')(texts)
    # text_enc = Dense(10000, activation='relu')(texts)
    # text_enc = Dense(5000, activation='relu')(texts)
    # text_enc = Dense(1000, activation='relu')(texts)
    
    # text_enc = Dense(d, activation='relu')(text_1)
    
    text_enc = Dense(d, activation='relu')(text_0)
    
    # text_enc = Dense(d * max_length, activation='relu')(text_0)
    # reshaped_text_enc = Reshape((max_length, d))(text_enc)
    
    # reshaped_text_enc = Reshape((d,))(text_enc)
    
    # conc = Concatenate(axis=-1)([fused_emb, text_enc, demo_enc])
    conc = Concatenate(axis=-1)([fused_emb, text_enc, demo_enc])
    fore_op = Dense(V)(conc)
    op = Dense(1, activation='sigmoid')(fore_op)
    model = Model([demo, times, values, varis, texts], op)
    if forecast:
        fore_model = Model([demo, times, values, varis, texts], fore_op)
        return [model, fore_model]
    return model

# To tune:
# 1. Transformer parameters. (N, h, dropout)
# 2. Normalization

In [15]:
D = 2
max_len = 880
V = 134

In [16]:
repeats = {k:10 for k in [10,20,30,40,50,60]}
lds = [50, 60]
batch_size, lr, patience = 32, 0.0005, 10
d, N, he, dropout = 50,2,4,0.2
fore_savepath = 'Exp_GloVe/M/models/forecasting/forecasting_109_epochs.h5'

train_inds = np.arange(len(train_op))
valid_inds = np.arange(len(valid_op))
gen_res = {}

np.random.seed(2023)
for ld in lds:
    np.random.shuffle(train_inds)
    np.random.shuffle(valid_inds)
    train_starts = [int(i) for i in np.linspace(0, len(train_inds)-int(ld*len(train_inds)/100), repeats[ld])]
    valid_starts = [int(i) for i in np.linspace(0, len(valid_inds)-int(ld*len(valid_inds)/100), repeats[ld])]
    all_test_res = []
    for i in range(repeats[ld]):
        print ('Repeat', i, 'ld', ld)
        # Get train and validation data.
        curr_train_ind = train_inds[np.arange(train_starts[i], train_starts[i]+int(ld*len(train_inds)/100))]
        curr_valid_ind = valid_inds[np.arange(valid_starts[i], valid_starts[i]+int(ld*len(valid_inds)/100))]
        curr_train_ip = [ip[curr_train_ind] for ip in train_ip]
        curr_valid_ip = [ip[curr_valid_ind] for ip in valid_ip]
        curr_train_op = train_op[curr_train_ind]
        curr_valid_op = valid_op[curr_valid_ind]
        print ('Num train:',len(curr_train_op),'Num valid:',len(curr_valid_op))
        # Construct save_path.
        savepath = 'CLS/GloVe/M/models/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat'+str(i)+'_'+str(ld)+'ld'+'.h5'
        print (savepath)
        try:
            os.remove(savepath)
        except OSError:
            pass
        # Build and compile model.
        model, fore_model =  build_strats(D, max_len, V, d, N, he, dropout, forecast=True)
        model.compile(loss=mortality_loss, optimizer=Adam(lr))
        fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))
        # Load pretrained weights here.
        fore_model.load_weights(fore_savepath)
        # Train model.
        es = EarlyStopping(monitor='custom_metric', patience=patience, mode='max', 
                           restore_best_weights=True)
        cus = CustomCallback(validation_data=(curr_valid_ip, curr_valid_op), batch_size=batch_size)
        his = model.fit(curr_train_ip, curr_train_op, batch_size=batch_size, epochs=1000,
                        verbose=1, callbacks=[cus, es]).history
        model.save_weights(savepath)
#         # Test and write to log.
#         rocauc, prauc, minrp = get_res(test_op, model.predict(test_ip, verbose=0, batch_size=batch_size))
#         f.write(str(np.min(his['custom_metric']))+str(rocauc)+str(prauc)+str(minrp)+savepath+'\n')
#         print ('Test res', rocauc, prauc, minrp)
#         all_test_res.append([rocauc, prauc, minrp])
        
#     gen_res[ld] = []
#     for i in range(len(all_test_res[0])):
#         nums = [test_res[i] for test_res in all_test_res]
#         gen_res[ld].append((np.mean(nums), np.std(nums)))
#     print ('gen_res', gen_res)


# # # save to local
# # log_path = '/content/log.csv'
# # files.download(log_path)

Repeat 0 ld 50
Num train: 18275 Num valid: 4631
CLS/GloVe/M/models/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat0_50ld.h5


2024-06-14 08:41:39.714129: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31117 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:3a:00.0, compute capability: 7.0
2024-06-14 08:41:39.714807: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 31117 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB, pci bus id: 0000:3b:00.0, compute capability: 7.0
2024-06-14 08:41:39.715336: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 31117 MB memory:  -> device: 2, name: Tesla V100-SXM2-32GB, pci bus id: 0000:b2:00.0, compute capability: 7.0
2024-06-14 08:41:39.715884: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 31117 MB memory:  -> device: 3, name: Tesla V100-SXM2-32GB, pci bus id

Epoch 1/1000


2024-06-14 08:41:56.541139: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x14c6b1bae330 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-14 08:41:56.541180: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2024-06-14 08:41:56.541187: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (1): Tesla V100-SXM2-32GB, Compute Capability 7.0
2024-06-14 08:41:56.541192: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (2): Tesla V100-SXM2-32GB, Compute Capability 7.0
2024-06-14 08:41:56.541197: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (3): Tesla V100-SXM2-32GB, Compute Capability 7.0
2024-06-14 08:41:56.552598: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-14 08:

572/572 [==============================] - 46s 57ms/step - loss: 0.4760 - custom_metric: 1.3496
Epoch 2/1000
572/572 [==============================] - 29s 50ms/step - loss: 0.4179 - custom_metric: 1.3934
Epoch 3/1000
572/572 [==============================] - 29s 50ms/step - loss: 0.3876 - custom_metric: 1.4266
Epoch 4/1000
572/572 [==============================] - 28s 50ms/step - loss: 0.3709 - custom_metric: 1.4049
Epoch 5/1000
572/572 [==============================] - 29s 50ms/step - loss: 0.3491 - custom_metric: 1.3834
Epoch 6/1000
572/572 [==============================] - 28s 50ms/step - loss: 0.3352 - custom_metric: 1.3765
Epoch 7/1000
572/572 [==============================] - 39s 54ms/step - loss: 0.4702 - custom_metric: 1.4073
Epoch 2/1000
572/572 [==============================] - 29s 50ms/step - loss: 0.4189 - custom_metric: 1.4293
Epoch 3/1000
572/572 [==============================] - 29s 51ms/step - loss: 0.3952 - custom_metric: 1.4637
Epoch 4/1000
572/572 [==========

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



686/686 [==============================] - 34s 49ms/step - loss: 0.2390 - custom_metric: 1.3985
Epoch 14/1000
686/686 [==============================] - 34s 50ms/step - loss: 0.2206 - custom_metric: 1.3623
Repeat 5 ld 60
Num train: 21930 Num valid: 5557
CLS/GloVe/M/models/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat5_60ld.h5
Epoch 1/1000
686/686 [==============================] - 45s 54ms/step - loss: 0.4655 - custom_metric: 1.4003
Epoch 2/1000
686/686 [==============================] - 35s 50ms/step - loss: 0.4139 - custom_metric: 1.4480
Epoch 3/1000
686/686 [==============================] - 34s 50ms/step - loss: 0.3942 - custom_metric: 1.4479
Epoch 4/1000
686/686 [==============================] - 34s 50ms/step - loss: 0.3787 - custom_metric: 1.4613
Epoch 5/1000
686/686 [==============================] - 34s 50ms/step - loss: 0.3642 - custom_metric: 1.4386
Epoch 6/1000
686/686 [==============================] - 34s 50ms/step - loss: 0.3518 - custom_metric: 1.4445
Epoch 7/1000


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [17]:
1

1